In [1]:
from langchain_community.document_loaders import (
    PyPDFLoader,
    PDFMinerLoader,
    PyMuPDFLoader,
    UnstructuredPDFLoader,
)


c:\Users\Bengulore Hemanth\Desktop\RAG\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
try:
    pypdf_loader =PyPDFLoader("data/pdf/attention.pdf")
    pyPyPDF_docs=pypdf_loader.load()
    print(pyPyPDF_docs)
except Exception as e :
    print(f"error:{e}")
print(len(pyPyPDF_docs[0].page_content))

In [7]:
try:
    pymupdf_loader = PyMuPDFLoader("data/pdf/attention.pdf")
    pymupdf_docs = pymupdf_loader.load()

    print(f" Loaded {len(pymupdf_docs)} pages")
    print(f" Includes detailed metadata")
    print(pymupdf_docs)

except Exception as e:
    print(f" Error: {e}")

MuPDF error: library error: zlib error: invalid code lengths set

MuPDF error: library error: zlib error: invalid code lengths set

MuPDF error: library error: zlib error: invalid code lengths set

MuPDF error: library error: zlib error: invalid code lengths set

MuPDF error: library error: zlib error: incorrect header check

MuPDF error: library error: zlib error: incorrect header check

MuPDF error: library error: zlib error: incorrect header check

MuPDF error: library error: zlib error: incorrect header check

MuPDF error: library error: zlib error: invalid code lengths set

MuPDF error: library error: zlib error: invalid code lengths set

MuPDF error: library error: zlib error: invalid code lengths set

MuPDF error: library error: zlib error: invalid code lengths set

MuPDF error: library error: zlib error: incorrect header check

MuPDF error: library error: zlib error: incorrect header check

MuPDF error: library error: zlib error: incorrect header check

 Loaded 15 pages
 Includ

In [ ]:
from langchain_community.document_loaders import PyPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from typing import List
from langchain_core.documents import Document

In [ ]:
class smart_pdf_cleaner:
    def __init__(self,chunk_size =1000,chunk_overlap=100):
        self.chunk_size =chunk_size,
        self.chunk_overlap =chunk_overlap,
        self.text_splitter =RecursiveCharacterTextSplitter(
            chunk_size=chunk_size,
            chunk_overlap=chunk_overlap,
            separators=[" "]
        )
    def process_pdf(self, pdf_path: str) -> List[Document]:
        